## **Import Utilities**

In [ ]:
import pandas as pd
import time
import anthropic
import sys
import os

sys.path.append("..")  # Add the parent directory of LLM_Evaluations to the Python path
from llm_evaluation_utils import load_responses_df, \
                        check_and_store_response,   \
                        build_question_prompt,      \
                        QUESTIONS, QUESTION_HEAD, QUESTION_TAIL

api_key = os.environ.get("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=api_key)

model_name = "claude-3-sonnet-20240229"
max_tokens = 200        # Max generated tokens
temperature = 0.8


## Test

In [ ]:
transcript = "Here you can see the anatomy of a normal, uninjured thumb. The synovial tendon sheath and the tendon are clearly shown. Inflammation of the tendon sheath is known as stenosing tinosinovitis. Stenosing tinosinovitis of the thumb, often referred to as trigger thumb, is one of the more common abnormalities of the hand. This condition is associated with painful triggering or locking of the thumb. When the patient tries to straighten the thumb, the nodule jams beneath the pulley proximally. This occurs with a painful snap, much like a trigger being pulled and released, with either flexion or extension of the thumb. Small changes in the tendon's diameter will have a significant effect on the function of the thumb. The nodule is usually pulled proximally to the pulley with active thumb flexion. That includes anti-inflammatory medication and injection. Inject into the sheath, not the tendon. Put the needle obliquely towards the tendon. Withdraw the needle slightly and inject freely into the sheath. Ultrasound guided injection of the tendon sheath and pulley may be of benefit. The needle is inserted into the sheath above the tendon and fluid is injected. If symptoms persist, consider surgery. Release the pulley to allow active motion without triggering. Note the position of the radial digital nerve close to the incision. Everything seems to be fine with this thumb, but there's definitely something tricky going on with this one. Hello doctor, got something crazy going on with my thumb. Can you take a look at it? Come right away. All my videos and this video is for educational purposes only. Please consult your doctor before you make any decision about your medical care."

# system = " ".join([QUESTION_HEAD, QUESTION_TAIL])
# user = f"Question: {QUESTIONS[4]}\nTranscript: {transcript}"
prompt = f"""{QUESTION_HEAD} {QUESTIONS[4]} {QUESTION_HEAD}
Transcript: {transcript}"""

message = client.messages.create(
    model=model_name,
    max_tokens=250,
    temperature=0.1,
    # system=system,
    messages=[
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Score: "}
    ]
)

print(message.content[0].text)

## **Claude Responses**

### Load Data

In [ ]:
transcripts_dir = "../../Getting_Transcripts"
transcripts_file_name = "merged_filtered_videos_transcripts.csv"
responses_dir = "../../../Results/LLMs_Responses"

prompt_type = "COT_prompting"
topics = "last_3_topics"
results_file_name = f"{model_name}-{topics}-{prompt_type}"

responses_df = load_responses_df(transcripts_dir, transcripts_file_name, responses_dir, results_file_name)

print("responses_df shape:", responses_df.shape)
responses_df.head(2)

In [ ]:
experts_file = "../../../Videos_and_DISCERN_data/filtered_experts_scores.csv"
experts_df = pd.read_csv(experts_file)

responses_df.insert(2, "Topic", experts_df["Topic"])
responses_df = responses_df[responses_df['Topic'].isin(["Cluster Headache", "Trigger Finger", "Pudendal Nerve"])]
# col_to_drop = [f"Q{i}" for i in range(6,16)] + [f"Response_{i}" for i in range(6,16)]
# responses_df.drop(columns=col_to_drop, inplace=True)
print("responses_df shape:", responses_df.shape)
responses_df.head(2)

### Get Claude API Responses

In [ ]:
def query_model(client, model_name, prompt, max_tokens=250, temperature=0.8):
    """Sends a prompt to the specified large language model and returns its response."""
    response = client.messages.create(
        model=model_name,
        max_tokens=max_tokens,
        temperature=temperature,
        # system=system,
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "Score:"}      # tell the LLM what to start with 
        ]
    )
    return response.content[0].text

print_response = False

# Calculate the delay based on your rate limit
requests_limit_per_minute = 50
delay = 60.0 / requests_limit_per_minute

for index, row in responses_df.iterrows():
    video_id = row["Video ID"]
    transcript = row["Transcript"]
    print(f"Started with video ID: {video_id} | Index: {index}")

    for question_num in range(1, len(QUESTIONS) + 1):
        column_name = f"Response_{question_num}"
        if pd.isna(row[column_name]):
            prompt = build_question_prompt(transcript, question_num)
            
            response = query_model(client, model_name, prompt, max_tokens, temperature)
            check_and_store_response(response, responses_df, video_id, question_num, 
                                     rating_scale=5, remove_prompt=False, print_response=print_response)
            
            time.sleep(delay)

### Explore Results

In [ ]:
display_from_index = 45
responses_df.iloc[display_from_index:,3:18].head(1)

In [ ]:
columns_with_none = (responses_df.isna() | responses_df.isnull()).sum()
columns_with_none

In [ ]:
rows_with_none = responses_df[responses_df.isnull().any(axis=1)]
rows_with_none

In [ ]:
indices_with_problems = responses_df[responses_df["Problem"].apply(lambda x: len(x) > 0)].index.tolist()
print(indices_with_problems)

In [ ]:
from IPython.display import display, HTML

if indices_with_problems:
    index_with_problem = 70
    responses_with_problem_list = list(responses_df.loc[index_with_problem, 'Problem'])
    print(responses_with_problem_list)

    response_with_problem = responses_with_problem_list[0]
    text = responses_df.loc[index_with_problem, f"Response_{response_with_problem}"]
    display(HTML("<div style='white-space: pre-wrap;'>{}</div>".format(text)))

In [ ]:
# display the full responses for a specific transcript
index_to_display = 45
for question_num in range(1, 16):
    print(f"Q{question_num}:", responses_df.at[index_to_display,f"Response_{question_num}"])

### Store Results in a CSV File

In [ ]:
csv_output_file = os.path.join(responses_dir, f"{results_file_name}-response.csv")

responses_df.to_csv(csv_output_file, index=False, encoding="utf-8")